# Import Module

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import timm
import time

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
CFG={'BATCH_SIZE' : 128,
     'LR' : 0.001,
     'EPOCHS' : 100}

# Define GPU

In [3]:
if torch.cuda.is_available():
    device="cuda"
else:
    device="cpu"
    
print(f"device  : {device}")

device  : cuda


# SEED

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED']=str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic=True
    torch.backends.cudnn.benchmark=True

# Data

Define Class

In [5]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Directly Ensemble

In [6]:
class ResNet18(nn.Module):
    def __init__(self, classes=10, pretrained=True) -> None:
        super().__init__()
        self.model=timm.create_model('resnet18', pretrained=pretrained)

        input_features=self.model.fc.in_features
        self.model.fc=nn.Linear(input_features, classes, bias=True)

    def forward(self, x):
        x=self.model(x)
        return x

In [7]:
def train_loop(epoch, model, dataloader, optimizer, loss_fn):
    trainloss=[]
    trainacc=[]
    size=len(dataloader.dataset)
    for e in range(epoch):
        print(f"--------------------Start {e+1}--------------------")
        start=time.time()

        train_loss=0
        correct=0
        total=0

        model.train()

        for batch, data in enumerate(dataloader):
            X=data[0].to(device)
            y=data[1].to(device)

            optimizer.zero_grad()

            outputs=model(X)
            loss=loss_fn(outputs, y)
            loss.backward()
            optimizer.step()
            
            train_loss+=loss.item()
            _, predicted=outputs.max(1)
          
            total+=y.size(0)
            correct+=predicted.eq(y).sum().item()

            if batch%100==0:
                current=batch*len(X)
                print(f"Batch {batch} loss : {(train_loss/size):>0.3f} [{current:>5d}/{size:>5d}]")

        trainloss.append(train_loss/size)
        trainacc.append((100*correct/size))

        end=time.time()
        print(f"Train\n ACC : {(100*correct/size):>0.1f}%,  LOSS : {(train_loss/size):>0.3f}")
        print(f"Epoch Time : {(end-start) // 60} 분 {((end-start)%60):>0.1f} 초")
        print(f"--------------------End {e+1}--------------------\n\n")

    return trainloss, trainacc

In [8]:
model_num=5
for i in range(model_num):
    seed=i
    seed_everything(seed)

    degree=random.randint(1, 180)
    train_transform = transforms.Compose([
    #transforms.AugMix(),
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    transforms.RandomHorizontalFlip(),
    #transforms.ColorJitter(brightness=0.5),
    #transforms.RandomVerticalFlip(),
    #transforms.RandomRotation(degrees=degree)
    ])

    trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=train_transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=CFG['BATCH_SIZE'],
                                            shuffle=True, num_workers=2)
    
    model=ResNet18().to(device)
    loss_fn = nn.CrossEntropyLoss().to(device)

    #base_optimizer = optim.SGD
    optimizer=optim.Adam(model.parameters(),lr=CFG['LR'])

    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)
    print(f"{seed} Seed Training Start {degree}")
    train_loss, train_acc=train_loop(CFG['EPOCHS'], model, trainloader, optimizer, loss_fn)

    path='D:\\Assignment\\'
    torch.save(model.state_dict(), path+'cifar10_practice_seed_0530' + str(i) + '.pt')
    print(f"{seed} Seed Training done")
    print()

Files already downloaded and verified
0 Seed Training Start 99
--------------------Start 1--------------------
Batch 0 loss : 0.000 [    0/50000]
Batch 100 loss : 0.002 [12800/50000]
Batch 200 loss : 0.003 [25600/50000]
Batch 300 loss : 0.003 [38400/50000]
Train
 ACC : 81.8%,  LOSS : 0.004
Epoch Time : 1.0 분 3.9 초
--------------------End 1--------------------


--------------------Start 2--------------------
Batch 0 loss : 0.000 [    0/50000]
Batch 100 loss : 0.001 [12800/50000]
Batch 200 loss : 0.001 [25600/50000]
Batch 300 loss : 0.002 [38400/50000]
Train
 ACC : 88.8%,  LOSS : 0.003
Epoch Time : 0.0 분 57.7 초
--------------------End 2--------------------


--------------------Start 3--------------------
Batch 0 loss : 0.000 [    0/50000]
Batch 100 loss : 0.001 [12800/50000]
Batch 200 loss : 0.001 [25600/50000]
Batch 300 loss : 0.002 [38400/50000]
Train
 ACC : 91.2%,  LOSS : 0.002
Epoch Time : 0.0 분 57.4 초
--------------------End 3--------------------


--------------------Start 4-----

# Test

Load Model

In [14]:
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.TenCrop(224),
    transforms.Lambda(lambda crops: torch.stack([transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))(transforms.ToTensor()(crop)) for crop in crops]))
])
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                    download=True, transform=test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                    shuffle=False, num_workers=0)

Files already downloaded and verified


In [18]:
models=[]
for i in range(5):
    model = ResNet18()
    path='D:\\Assignment\\'
    model.load_state_dict(torch.load(path+'cifar10_practice_seed_0530' + str(i) +'.pt'))
    model.eval()
    model=model.to(device)
    models.append(model)

Pred

In [19]:
# def test_loop(model, dataloader):
#     correct=0
   
#     size=len(dataloader.dataset)

#     model.eval()

#     with torch.no_grad():
#         for batch, data in enumerate(dataloader):
#             X=data[0].to(device).float()
#             y=data[1].to(device).float()
            
#             outputs=model(X)

#             _, predicted=torch.max(outputs.data, 1)
#             correct+=(predicted == y).sum().item()
#     print(f'Accuracy of the network on the 10000 test images: {(100 * (correct / size)):>0.2f} %')

In [20]:
# test_loop(model, testloader)

In [21]:
correct = 0
total = 0
cnt=0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)  # Move the input data to the GPU
        bs, ncrops, c, h, w = images.size()       
        outputs = torch.zeros(bs, 10).to(device)  # Initialize the output tensor with zeros
        for model in models:
            model_output = model(images.view(-1, c, h, w))  # Reshape the input to (bs*10, c, h, w)
            model_output = model_output.view(bs, ncrops, -1).mean(1)  # Average the predictions of the 10 crops
            outputs += model_output
            cnt+=1
            #print(f"{cnt} model finish")
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the ensemble on the 10000 test images: %f %%' % (100 * correct / total))

Accuracy of the ensemble on the 10000 test images: 97.490000 %


In [22]:
# 제일 좋은거 lr 0,1 합친거 => 그냥 baseline임